# Epic Seven Gear Simulator Testing Documentation
This notebook documents the testing process.

* The json data files are stored in the `/data` folder. The scripts to prepare these json file are also in the `/data` folder.
* The test modules (along with this notebook) are stored in the `/tests` folder.
* The class and function modules for the app are stored in the `/src` folder.
* `unittest` module is used for running the tests.

### Directory Structure Issue

Modules in `/tests` folder have to import data and modules from the `/data` or `/src` folders respectively, so we need to add the parent directory to the path.

This is solved by importing `set_directory` function from the `set_directory_function` module. As such, every testing module should have the following two lines of code at the top, so that the parent folder can be added to the path.

In [ ]:
from set_directory_function import set_directory
set_directory()

### Running the tests
The tests must be run from the root directory as:

python tests/test_module_name.py

e.g.,
python tests/test_get_stat_by_id.py

## Testing Validation Functions
In many of the functions that follow, the inputs need to be validated, so this section tests whether these validation functions work as intended.

### validate_stat_id()
Source: [validation_utils.py](https://github.com/mesaqlain/e7_items/blob/main/src/validation_utils.py)

Function: 
* Raises a ValueError input is None
* Raises a ValueError if input is not str. 
* Acceptable parameters: 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10 (as of now, if more stats are added this could increase later). Parameter can be entered as str or int.

Returns: str(stat_id)

Test Module: [test_validate_stat_id.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_validate_stat_id.py)

Test Plan:
* stat_id = '0', '1', '2', '3', '4', '5', '6', '7', '8', '9'. '10' (exactly as written in STATS file, should work)
* stat_id = 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10 (ints, should work)
* stat_id = -1 (some negative number, should be error)
* stat_id = 'Health' (some string that's not a number)
* stat_id = 111 (some large number as int that is not in acceptable parameter)
* stat_id = '111' (some large number as str that is not in acceptable parameter)
* stat_id = None (should raise ValueError)

Result: ALL PASSED.

### validate_gear_type()
Source: [validation_utils.py](https://github.com/mesaqlain/e7_items/blob/main/src/validation_utils.py)

Function: 
* Returns None if input is None
* Raises a ValueError if input is not str. 
* Acceptable parameters: 'Weapon', 'Helm', 'Armor', 'Necklace', 'Ring', 'Boots'. Parameters are converted to lower case before validating, so lower, upper or mixed case is acceptable.

Returns: gear_type.lower()

Test Module: [test_validate_gear_type.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_validate_gear_type.py)

Test Plan:
* gear_type = 'Weapon', 'Helm', 'Armor', 'Necklace', 'Ring', 'Boots' (exactly as written in TYPES file, should work)
* gear_type = 'weapon', 'helm', 'armor', 'necklace', 'ring', 'boots' (all lower case should still work)
* gear_type = 'weapOn', 'hElm', 'arMor', 'nEcklacE', 'rINg', 'BOOTS' (mix of lower and upper case, should work)
* gear_type = 'Sword', 'mask', 'MAIL', 'Jewelry', 'Accessory', 'Shoes' (random strings, should raise error)
* gear_type = 1 (non string parameter, should raise ValueError)
* gear_type = None (should return None)

Results: ALL PASSED.

### validate_stat_type(stat_type, mod = False)
Source: [validation_utils.py](https://github.com/mesaqlain/e7_items/blob/main/src/validation_utils.py)

Function: 
* Summary: Validates stat_type input and returns stat_type.lower()
* Defaults to 'mainstat' if input is None. mod default is False (used in parse_stat method)
* Raises a ValueError if input is not str. 
* Raises a ValueError if input is 'mainstat' and mod is True.
* Raises a ValueError if input is 'mainstat' and rolled is not equal to 0.
* Acceptable parameters: 'mainstat' and 'substat'. Parameters are converted to lower case before validating, so lower, upper or mixed case is acceptable.

Returns: stat_type.lower()

Test Module: [test_validate_stat_type.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_validate_stat_type.py)

Test Plan:
* valid inputs (mod default False)
    * stat_type = 'mainstat', 'substat', 'Mainstat', 'Substat', 'MainSTAT', 'SUBstAt'
* invalid inputs (mod default False)
    * stat_type =  'mainstats', 'substats', 'subs', 'main', 'stats', 
    * stat_type = 1
    * stat_type = None
* valid inputs (mod True)
    * stat_type = substat', 'Substat','SUBstAt'
* invalid inputs (mod True)
    * stat_type = 'mainstat', Mainstat', 'MainSTAT'
    
* valid inputs 
    * rolled = 0 and 'mainstat' PASSED
    * rolled = 2 and 'substat' PASSED

* invalid inputs
    * rolled = 3 and 'mainstat' PASSED

Results: ALL PASSED.

### is_valid_stat_entry(stat_entry)
Source: [validation_utils.py](https://github.com/mesaqlain/e7_items/blob/main/src/validation_utils.py)

Function: 
* Summary: Checks whether an entry is a valid entry from the stats.json file.
* Checks whether the entry is a dict, if it has the keys 'id', 'text', and 'key_stat'
* If all conditions fulfilled, returns True
* If any of the conditions not fulfilled, returns False
* Acceptable parameters: Anything

Returns: boolean

Test Module: [test_is_valid_stat_entry.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_is_valid_stat_entry.py)

Test Plan:
* valid entries
    * stat_entry = all entries in stats.json (check all) PASSED
* invalid entries (integer, str, list, )
    * stat_entry =  1, 'stats', [1, 2, "stats"], None. PASSED
    * stat_entry = some random dict PASSED
    * stat_entry = some random dict containing an id key, an id key and text key PASSED

Results: ALL PASSED

### validate_selected_stats(selected_stats)
Source: [validation_utils.py](https://github.com/mesaqlain/e7_items/blob/main/src/validation_utils.py)

Function: 
* Summary: Accepts a list which can be empty or contain only valid id's.
* Checks whether the entry None or empty list : return empty list []
* Checks whether the entry is a list, if not raises ValueError
* Checks whether all the entries in the list are valid stat id's or not (by checking against keys from stats.json)
* Acceptable parameters: List, valid stat id's inside the list, empty list or None (both return empty list)

Returns: empty list or list containing valid stat objects

Test Module: [test_validate_selected_stats.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_validate_selected_stats.py)

Test Plan:
* valid entries
    * selected_stats = [0], [0, 1], multiple. PASSED
    * selected_stats = ['3'], ['4', '10'], multiple. PASSED
    * selected_stats =  ['2', 8], multiple. PASSED

* invalid entries (integer, str, list, )
    * selected_stats = [], None. PASSED
    * selected_stats =  1, 'stats', [1, 2, 'health'], [{}, {'id' : 1}] PASSED

Results: ALL PASSED

### validate_rolled(rolled)
Source: [validation_utils.py](https://github.com/mesaqlain/e7_items/blob/main/src/validation_utils.py)

Function: 
* Summary: Validates whether 'rolled' input is an int in the range(0,6).
* Check whether entry is None: returns 0 by default
* Checks whether entry is int and whether it's in the range (0,6)
* Checks whether entry is 'mainstat', if it is, rolled cannot be anything but 0.
* Acceptable parameters: int [0, 1, 2, 3, 4, 5]

Returns: int in range(0, 6)

Test Module: [test_validate_rolled.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_validate_rolled.py)

Test Plan:
* valid entries
    * rolled = [0, 1, 2, 3, 4, 5, 6]
    * repeat above, specifying stat_type = 'substat'
* invalid entries (None, str, list, negative, large int)
    * repeat above, removing the 0 and specifying stat_type = 'mainstat' (mainstats cannot have non-zero rolls)
    * rolled = None
    * rolled =  ['one', [1, 2], -1, 1000]

Results: 

### validate_mod(mod)
Source: [validation_utils.py](https://github.com/mesaqlain/e7_items/blob/main/src/validation_utils.py)

Function: 
* Summary: Validates the mod parameter. It should be set to True when we're parsing a stat that is modded
* Check whether entry is bool: returns ValueError if it isn't a bool
* Aacceptable parameters: bool

Returns: bool

Test Module: [test_validate_mod.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_validate_mod.py)

Test Plan:
* valid entries
    * mod = [True, False]
* invalid entries
    * mod = None, str, list, negative, large int, 0, 1 (since we're testing for bool, 0 and 1 won't work)

Results: ALL PASSED

### validate_mod_type(mod_type)
Source: [validation_utils.py](https://github.com/mesaqlain/e7_items/blob/main/src/validation_utils.py)

Function: 
* Summary: Validates the mod_type parameter. Used for modification system.
* If None provided, default to 'greater'
* Raises ValueError if it is not a str and if it isn't 'greater' or 'lesser' (not case sensitive)
* Aacceptable parameters: (str) 'greater' or 'lesser'

Returns: (str) 'greater' (default) or 'lesser'

Test Module: [test_validate_mod_type.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_validate_mod_type.py)

Test Plan:
* valid entries
    * mod_type = ['greater', 'Greater', 'GreaTER']
    * mod_type = ['lesser', 'Lesser', 'LeSSer']
    * mod_type = None (should return 'greater')

* invalid entries
    * mod = some str, list, negative int, large int 

Results: 


### validate_gear_grade(gear_grade)

Source: [utilities.py](https://github.com/mesaqlain/e7_items/blob/main/src/utilities.py)

Function: Validates whether the input gear grade is valid. Acceptable parameters are: ['normal', 'good', 'rare', 'heroic', 'epic']

Test Module: [test_validate_gear_grade.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_validate_gear_grade.py)

Test Plan: 
* valid inputs: 
    * ['normal', 'good', 'rare', 'heroic', 'epic'] (lower case)
    * ['Normal', 'Good', 'Rare', 'Heroic', 'Epic'] (Capitalized)
    * ['NORMAL', 'GOOD', 'RARE', 'HEROIC', 'EPIC'] (upper)
    * ['norMAL', 'GooD', 'rARe', 'HeRoIc', 'ePIC'] (mixed)
* invalid inputs: 
    * ['some str', 1, -1, 0]
* none input: 
    * Generate 1000 gear_grades and check whether they are in ['normal', good', 'rare', 'heroic', 'epic']

Results: ALL PASSED

### validate_gear_level(gear_level)

Source: [utilities.py](https://github.com/mesaqlain/e7_items/blob/main/src/utilities.py)

Function: Validates whether the input gear level is a valid gear level. If none provided, default to 85. Accept int only and in the range(58, 101)

Test Module: [test_validate_gear_level.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_validate_gear_level.py)

Test Plan: 
* valid inputs: [58, 69, 71, 85, 88, 90, 100]
* invalid inputs: [56, 57, 101, 102, -1. '70', 'level two', [], {}]
* none input: None (should return 85)


Results: ALL PASSED

## Testing Utility Functions

### get_random_grade()

Source: [utilities.py](https://github.com/mesaqlain/e7_items/blob/main/src/utilities.py)

Function: Gets a random gear grade - 'rare', 'heroic' or 'epic'. The rates of choosing them are 0.35, 0.53, and 0.12 respectively (rates taken from inside game and can be modified later in (prep_data_GRADE.py)[https://github.com/mesaqlain/e7_items/blob/main/data/prep_data_GRADE.py] file. No arguments needed.

Test Module: [test_get_random_grade.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_get_random_grade.py)

Test Plan: 
* Test whether the result we get is in ['normal', good', 'rare', 'heroic', 'epic']
* Generate 100,000 grades and test whether the results are as expected (as in if we get rare 35% of the time, heroic 53% of the time, epic 12% of the time, and the other two 0% of the time).

Test Implementation:
* Initialize starting variables and set value to 0.
* Use get_random_grade to pick a grade and increment the corresponding grade variable value by 1
* Repeat above 100,000 times.
* Get simulated rates
* Use assertAlmostEqual with delta value (tolerance level) of 0.02 to check whether the difference between actual and simulated rates is between 0.02.

Results: ALL PASSED

### get_gear_tier(gear_level):

Source: [utilities.py](https://github.com/mesaqlain/e7_items/blob/main/src/utilities.py)

Function: Gets the tier of the gear based on the provided gear level. 
* Lvl 58-71: Tier 5
* Lvl 72-85: Tier 6
* Lvl 86-100: Tier 7

Returns: int 5, 6 or 7.

Test Module: [test_get_gear_tier.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_get_gear_tier.py)

Test Plan: 
* valid inputs: 
    * 58-71: should return 5
    * 72-85: should return 6
    * 86-100: should return 7
    * None: should return 6
* invalid inputs: [56, 57, 101, 102, -1. '70', 'level two', [], {}] (should raise Value Error)

Results: ALL PASSED

### get_reforge_increase():

Source: [utilities.py](https://github.com/mesaqlain/e7_items/blob/main/src/utilities.py)

Function: 
* Summary: Get the value by which a stat will increase by when an item has been reforged.
* Validates inputs
* Gets an int value from corresponding 'mainstat' or 'substat' section and the index based on 'rolled' count.
* NOTE: Moved to utilities.py file from stats.py

Returns: (int) STATS[stat_id]['reforge'][stat_type][rolled]

Test Module: 
* [test_get_reforge_increase.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_get_reforge_increase.py)
* [test_get_reforge_increase_in_class.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_get_reforge_increase_in_class.py) (Not currently relevant as it has been moved)

Test Plan:

* Valid entries ALL PASSED
    * stat_id = 0, None, None (if stat_type is None then default is 'mainstat', if rolled is None, then default is 0). Expected = 525
    * stat_id = 0, 'mainstat', 0. Expected = 525
    * stat_id = 0, 'substat'. Expected = 11
    * stat_id = 5, 'substat', 3. Expected = 5
    * stat_id = '5', 'substat', 3. Expected = 5


* Invalid entries ALL PASSED
    * stat_id = -1, None, None
    * stat_id = 0, 'some string', 2
    * stat_id = 3, 'substat', -1
    * stat_id = 4, 'mainstat', '1'
    
* Test after integrating into class (same as above tests): ALL PASSED
    * def setUp(self) and tearDown(self)
    * Assign self.stat.stat_id and self.stat.stat_type values before testing the function        

Results: ALL PASSED

## Testing Stat Class

### get_stat_by_id()

Source: [stats.py](https://github.com/mesaqlain/e7_items/blob/main/src/stats.py)

Function: Retrieves data on a STAT based on the given id. It returns a dictionary.

Test Module: [test_get_stat_by_id.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_get_stat_by_id.py)

The Stat() class initiates with no data. Running the get_stat_by_id() method should return a dictionary from the STATS dictionary based on the id provided. The function should be able to handle both int and str arguments as long as the number is between 0 and 10. So we need to test the following cases.

Valid stat_id should return a dictionary containing stats where id is 0 or 7 (compare to STATS[0] and STATS[7]):
* stat_id = 0 
* stat_id = '0' 
* stat_id = 7 
* stat_id = '7' 
* stat_id = 0, but check whether the store selected_stat_id is correctly returned as 0.
* stat_id = '7' , but check whether the store selected_stat_id is correctly returned as 0.


Invalid stat_id should raise ValueError:
* stat_id = 100
* stat_id = '100'
* stat_id = -1 (negative case)
* stat_id = 1000000 (big number)
* stat_id = 'Health' (some random string)
* stat_id = None

Additional tests:
* Check whether we get the correct stat_id and stat_key attributes

Result: ALL PASSED

### get_random_stat()
Source: [stats.py](https://github.com/mesaqlain/e7_items/blob/main/src/stats.py)

Function: 
* Summary: Retrieves data on a random STAT chosen from the list of STATS in the stats.json file. 
* Validates gear_type input
* Initializes an empty list called pool
* If gear_type is None, the pool will contain all stats from the stats.json file.
* If gear_type is provided, the pool will be limited to the stats allowed according to stat_type (for more information on allowed stats on specific gear type, refer to prep_data_TYPES.py module or check Section 5.2 in the README file.)
* Chooses a random id from the pool and runs the get_stat_by_id() function to get that stat.

Returns: Stat dic

Test Module: [test_get_random_stat.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_get_random_stat.py)

Test Plan:
* get_random_stat() with no inputs (this means it should use 'mainstat' as default gear_type = None)
    * Check that we do get a dictionary that is one of the stats entries in stats.json PASSED 
    * Generate 1000 random stats and count how many times each stat was chosen, they should all have been chosen at least once. PASSED

* Valid stat_type inputs (while gear_type = None):
    * Repeat above test by specifying stat_type = 'mainstat'. PASSED
    * Repeat above test by specifying stat_type = 'substat'. PASSED

* Valid gear_type inputs (while stat_type = None):
    * Repeat above test by specifying gear_type = 'weapon' (we expect to see the stat_id = 0 only (because Weapons can only have that as mainstat)) PASSED
    * Repeat above test by specifying gear_type = 'helm' (we expect to see the stat_id = 2 only (because Helms can only have that as mainstat)) PASSED
    * Repeat above test by specifying gear_type = 'armor' (we expect to see the stat_id = 4 only (because Armor can only have that as mainstat)) PASSED
    * Repeat above test by specifying gear_type = 'necklace' (we expect to see no stat_id = 8,9,10 (because Necklace cannot have those as mainstat), while others should be chosen at least once)  PASSED
    * Repeat above test by specifying gear_type = 'ring' (we expect to see no stat_id = 6,7,10 (because Ring cannot have those as mainstat), while others should be chosen at least once) PASSED
    * Repeat above test by specifying gear_type = 'boots' (we expect to see no stat_id = 6,7,8,9 (because Boots cannot have those as mainstat), while others should be chosen at least once) PASSED
    
* Valid gear_type inputs (while stat_type = 'mainstat'):
    * Repeat all tests in the previous section by specifying stat_type = 'mainstat', we should see same results ALL PASSED

* Valid gear_type inputs (while stat_type = 'substat'):
    * Repeat above test by specifying gear_type = 'weapon' (we expect to see no stat_id = 0, 4, 5 (because Weapons cannot have those as substats), while others should be chosen at least once) PASSED
    * Repeat above test by specifying gear_type = 'helm' (we expect to see no stat_id = 2 (because Helms cannot have those as substats), while others should be chosen at least once) PASSED
    * Repeat above test by specifying gear_type = 'armor' (we expect to see no stat_id = 0, 1, 4 (because Armors cannot have those as substats), while others should be chosen at least once) PASSED
    * Repeat above test by specifying gear_type = 'necklace', 'ring', 'boots' (we expect see all stats chosen at least once) PASSED
    
* Invalid stat_type inputs (while gear_type = None):
    * stat_type = 111, '111', 'health' (test integers, random strings, etc) PASSED
    
* Invalid gear_type inputs (while stat_type = None):
    * gear_type = 111, '1', 'Sword' (test integers, random strings, etc) PASSED

* Invalid inputs for both:
    * gear_type = 111, stat_type = 'Sword' PASSED
    
* Additional Tests to check if correct selected_stat, stat_type, gear_type is returned
    * gear_type = 'ring', stat_type = 'mainstat' PASSED
    * gear_type = 'helm', stat_type = 'substat' PASSED

Test Plan Implementation:
* Create an empty set called selected_stat_ids = set()
* Generate 1000 random stats and take the 'id' of those stats and add it to selected_stat_ids
* Define expected_stat_ids, which contains the allowed stats for a particular gear type (based on stats)
* Calculate missing_stat_ids = expected_stat_ids - selected_stat_ids
* Calculate extra_stat_ids = selected_stat_ids - expected_stat_ids
* If missing_stat_ids set is empty, it means that all the expected stats were selected, and the assertion will pass.
* If extra_stat_ids set is empty, it means that no extra stats were selected, and the assertion will pass.

Further Test Implementation:
* As more Stat class attributes are added, if needed, they can be aded to the test.

Results: ALL PASSED

### get_non_overlapping_stat(selected_stats=[], stat_type='substat', gear_type=None):

Source: [stats.py](https://github.com/mesaqlain/e7_items/blob/main/src/stats.py)

Function: 
* Summary: Retrieves data on a random stat that is not already in the pool of 'selected stats'. 
* Validates inputs
* Gets a random stat
* If that random stat already exists in the selected pool, get another random stat.
* Repeat until a stat not already in the pool in selected.
* Return the new stat
* Chooses a random id from the pool and runs the get_stat_by_id() function to get that stat.

Returns: Stat dic

Test Module: 
* [test_get_non_overlapping_stat.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_get_non_overlapping_stat.py)
* [test_get_non_overlapping_stat_in_class.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_get_non_overlapping_stat_in_class.py)

Test Plan:

* Use empty list and None for selected stats PASSED
    * Check that we do get a dictionary that is one of the stats entries in stats.json  
    * Generate 1000 random stats and count how many times each stat was chosen, they should all have been chosen at least once if empty list was provided. 

* Valid selected_stats inputs: PASSED
    * We'll create a list with each of the STATS entries and generate 1000 non-overlapping stats, none of those stats should contain the one in the seleceted_list.
    * A list containing two chosen stats from stats.json, and another containing three.
    * We'll repeat the above test, making sure that the returned output is not the same as one of the selected stats.

* Valid gear_type inputs (while stat_type = 'substat'): PASSED
    * Repeat above test by specifying gear_type = 'weapon' (we expect to never see stats 0, 4, 5 + the stat that was in selected stats 
    * We won't test for any other gear_type because this was tested extensively for get_random_id().
    
* We repeat some of the tests from above by incorporating the function inside the Stat class PASSED

* Additional Test after adding selected_stat_id class attribute
    * Run some of the above valid tests and check whether the stored selected_stat_id is correct.
    
Results: ALL PASSED

### get_mod_value(stat_id, gear_level=85, rolled=None, mod_type='greater')

Source: 
* Testing: [test_functions.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_functions.py)
* Actual: [utilities.py](https://github.com/mesaqlain/e7_items/blob/main/src/utilities.py)

Function: 
* Summary: Choose a value for a modified stat based on given rolled_count, gear_level, and mod_type
* Validates inputs
* Get the list of mod values for the given criteria and mod_type
* Checks gear_level and rolled_count to get the list of values
* Randomly pick a value from the list

Returns: (int) value of stat

Test Module: 
* [test_get_mod_value.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_get_mod_value.py)

Test Plan:

* Valid entries ALL PASSED
    * specify stat_id = 0 (attack_flat) (leaving others blank, so default lv=85, mod_type=greater, rolled=0). We expect value between 33-47 
    * specify stat_id = 0 (attack_flat), lv=85 (others default). We expect value between 33-47 
    * specify stat_id = 0 (attack_flat), lv=85, rolled=0. We expect value between 33-47 
    * specify stat_id = 0 (attack_flat), lv=85, rolled=0, mod_type='greater'. We expect value between 33-47 
    * specify stat_id = 0 (attack_flat), lv=85, rolled=0, mod_type='lesse'. We expect value between 28-40
    * specify stat_id = 7 (crit_damage) (other arguments default) (we expect values 4-7)
    * specify stat_id = 6 (crit_chance), rolled=1 (other args default) (expected 3-6)
    * specify stat_id = 5 (defense_percent), rolled=2, mod_type='lesser' (expect 8-11)
    * specify stat_id = 10 (speed), rolled=4, mod_type='greater', gear_level=90 (expect 10-13)
    * specify stat_id = 2 (flat_health), rolled=5, mod_type='lesser', gear_level=88 (expect 477-560)

* Invalid entries ALL PASSED
    * stat_id = 1, gear_level=101 (invalid gear level)
    * stat_id = -1, (invalid stat_id)
    * stat_id = 10, mod_type = 'big mod' (invalid mod_type)
    * stat_id = 5, rolled = 6
    * stat_id = 5, rolled = -1
    
* Test implementation
    * Perform each test 10,000 times to check that every value has been picked at least once.
    
* Test after integrating into class


Results: 

### get_stat_value(stat_id, stat_type='mainstat', gear_level=85, gear_grade=None)

Source: 
* Testing: [test_functions.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_functions.py)
* In class: [utilities.py](https://github.com/mesaqlain/e7_items/blob/main/src/utilities.py)

Function: 
* Summary: Choose a value for a non-modified stat based on stat_id, stat_type, gear_level, and gear_grade
* Validates inputs and gets gear tier
* Get the list of stat values for the given criteria 
* If fixed (main), assigns value; if rand (sub), chooses a value based on rates

Returns: (int) value of stat

Test Module: 
* [test_get_stat_value.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_get_stat_value.py)

Test Plan:

* Valid entries 
    * specify stat_id = 0 (attack_flat) (leaving others blank, so default lv=85, 'mainstat', grade=random). We expect 100 (same val for all grades) 
    * specify stat_id = 0 (attack_flat), lv=85 (others default). We expect value 100 
    * specify stat_id = 0 (attack_flat), lv=85, 'mainstat'. We expect value 100  
    * specify stat_id = 0 (attack_flat), lv=85, 'mainstat', grade='rare'. We expect value 100  
    * specify stat_id = 0 (attack_flat), lv=90, 'mainstat', grade='rare'. We expect value 103  
    * specify stat_id = 7 (crit_damage) (other arguments default) (we expect value 13)
    * specify stat_id = 6 (crit_chance), lv=88, 'substat', grade = 'epic (we expect values 3-6)
    * specify stat_id = 5 (defense_percent), lv=70, 'substat', 'grade'='rare' (we expect values 4-6)
    * specify stat_id = 10 (speed), 'substat', gear_level=90, grade='epic' (expect 3-5)

* Invalid entries 
    * No need to check as we've thoroughly tested the input validation functions before
    
* Test implementation
    * Perform each test 10,000 times to check that every value has been picked at least once.
    
* Test after integrating into class

Results: ALL PASSED

### parse_stat(self, gear_grade=None, gear_level=85, mod=False, rolled=None, mod_type='greater')

Source: [stats.py](https://github.com/mesaqlain/e7_items/blob/main/src/stats.py)

Function: 
* Summary: Probably the most important method for the Stat class. It provides a fixed value for a stat based on its type ('substat' or 'mainstat') and given gear grade and level.
* Validates inputs and gets gear_tier from gear_level
* Assigns the following class attributes:
    * self.text 
    * self.gear_grade
    * self.gear_level 
    * self.gear_tier 
    * self.rolled 
* Checks if modded is True or False (default is False) and gets the appropriate stat value based on this condition
* If modded, sets self.modded to True (default is False)
* Further assigns the following attributes:
    * self.value 
    * self.value_key
    * self.reforge_increase 
    
Returns self

Test Module:
* [test_parse_stat.py](https://github.com/mesaqlain/e7_items/blob/main/tests/test_parse_stat.py)

Test Plan:
* Initialize 